## DATA INGESTION

In [1]:
## Document structure

from langchain_core.documents import Document

In [8]:
doc=Document(
page_content="This is the main context in order to craeate a chatbot-rag",
metadata={
    "source":"example.txt",
    "pages":1,
    "author":"shilpa",
    "date_created":"2025-10-17"
}

)
    

In [9]:
## create a simple txt file
import os
os.makedirs("data/text_files",exist_ok=True)

In [24]:
## create a content manually with a file dir
sample_texts = {
   "data/text_files/python_intro.txt" :"""
Python is a versatile, high-level programming language that emphasizes simplicity and readability. 
It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms 
such as procedural, object-oriented, and functional programming, making it suitable for a wide range of applications. 
Its extensive standard library, combined with a vast ecosystem of third-party packages, allows developers to perform 
tasks ranging from web development and automation to data analysis and artificial intelligence. 

Python’s clean syntax and dynamic typing make it beginner-friendly, while its scalability and integration capabilities 
make it powerful enough for complex enterprise applications. The language’s active global community continues to enhance 
its features and tools, ensuring Python remains one of the most widely used and trusted programming languages in the world.
""",


"data/text_files/machine_learning_intro.txt" : """
Machine Learning (ML) is a core branch of Artificial Intelligence (AI) that focuses on creating systems capable of 
automatically learning and improving from experience without being explicitly programmed. ML algorithms analyze 
large volumes of data to uncover patterns, make predictions, and support intelligent decision-making processes. 

The field of Machine Learning encompasses several learning paradigms such as supervised learning, unsupervised learning, 
and reinforcement learning. Its applications are vast and transformative — powering recommendation engines, predictive 
analytics, autonomous systems, natural language processing, and image recognition. 

Python plays a pivotal role in the Machine Learning ecosystem due to its simplicity and rich set of libraries 
like NumPy, Pandas, scikit-learn, TensorFlow, and PyTorch. These tools provide developers and researchers 
with efficient frameworks to build, train, and deploy robust machine learning models across diverse domains.
"""
}
for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("sample text files created")

sample text files created


In [1]:
## textloader
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

loader=TextLoader("data/text_files/python_intro.txt",encoding="utf-8")
document=loader.load()
print(document)
                  


[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='\nPython is a versatile, high-level programming language that emphasizes simplicity and readability. \nIt was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms \nsuch as procedural, object-oriented, and functional programming, making it suitable for a wide range of applications. \nIts extensive standard library, combined with a vast ecosystem of third-party packages, allows developers to perform \ntasks ranging from web development and automation to data analysis and artificial intelligence. \n\nPython’s clean syntax and dynamic typing make it beginner-friendly, while its scalability and integration capabilities \nmake it powerful enough for complex enterprise applications. The language’s active global community continues to enhance \nits features and tools, ensuring Python remains one of the most widely used and trusted programming languages in the worl

In [11]:
## Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory 
dir_loader=DirectoryLoader(
    "data/text_files",
    glob="**/*.txt",##pattern to match files
    loader_cls=TextLoader,##loader class to use
    loader_kwargs={'encoding':'utf-8'},
    show_progress=False
)
documents=dir_loader.load()
print(documents)

[Document(metadata={'source': 'data\\text_files\\machine_learning_intro.txt'}, page_content='\nMachine Learning (ML) is a core branch of Artificial Intelligence (AI) that focuses on creating systems capable of \nautomatically learning and improving from experience without being explicitly programmed. ML algorithms analyze \nlarge volumes of data to uncover patterns, make predictions, and support intelligent decision-making processes. \n\nThe field of Machine Learning encompasses several learning paradigms such as supervised learning, unsupervised learning, \nand reinforcement learning. Its applications are vast and transformative — powering recommendation engines, predictive \nanalytics, autonomous systems, natural language processing, and image recognition. \n\nPython plays a pivotal role in the Machine Learning ecosystem due to its simplicity and rich set of libraries \nlike NumPy, Pandas, scikit-learn, TensorFlow, and PyTorch. These tools provide developers and researchers \nwith ef

: 

In [4]:
## pdf Loader

from langchain_community.document_loaders import DirectoryLoader ,PyPDFLoader,PyMuPDFLoader

## load all the text files from the directory 
dir_loader=DirectoryLoader(
    "../data/pdf",
    glob="**/*.pdf",##pattern to match files
    loader_cls=PyMuPDFLoader,##loader class to use
   
    show_progress=False
)
pdf_documents=dir_loader.load()
print(pdf_documents)

[Document(metadata={'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '..\\data\\pdf\\HR-Policy (1).pdf', 'file_path': '..\\data\\pdf\\HR-Policy (1).pdf', 'total_pages': 18, 'format': 'PDF 1.4', 'title': 'HR-Policy', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content="HUMAN RESOURCE POLICY  \n \n \nINTRODUCTION  \nHuman resource policies are continuing guidelines on the approach of which an \norganization  intends to adopt in managing its people. They represent specific guidelines to \nHR managers  on various matters concerning employment and state the intent of the \norganization on  different aspects of Human Resource management such as recruitment, \npromotion,  compensation, training, selections etc. They therefore serve as a reference point \nwhen  human resources management practices are being developed or when decisions are \nbeing  made about an organ

### RAG PIPELINE DATA INGESTION TO VECTORDB PIPELINE


In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [5]:
## read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """process all pdf files in a directory"""
    all_documents=[]
    pdf_dir=Path(pdf_directory)

    ##find all pdf files recursively
    pdf_files=list(pdf_dir.glob("**/*.pdf"))
    print(f"Found{len(pdf_files)}PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nprocessing:{pdf_file.name}")
        try:
            loader=PyPDFLoader(str(pdf_file))
            documents=loader.load()

            #add source info to metadata
            for doc in documents:
                doc.metadata['source_file']=pdf_file.name
                doc.metadata['file_type']='pdf'

            all_documents.extend(documents)
            print(f"Loaded{len(documents)} pages")

        except Exception as e:
            print(f"error:{e}")
    
    print(f"\n total documents loaded:{len(all_documents)}")
    return all_documents
#process all PDFs in the data directory
all_pdf_documents=process_all_pdfs("../data")


Found1PDF files to process

processing:HR-Policy (1).pdf
Loaded18 pages

 total documents loaded:18


In [6]:
all_pdf_documents

[Document(metadata={'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'HR-Policy', 'source': '..\\data\\pdf\\HR-Policy (1).pdf', 'total_pages': 18, 'page': 0, 'page_label': '1', 'source_file': 'HR-Policy (1).pdf', 'file_type': 'pdf'}, page_content="HUMAN  RESOURCE  POLICY     INTRODUCTION   \nHuman  resource  policies  are  continuing  guidelines  on  the  approach  of  which  an  \norganization\n  \nintends\n \nto\n \nadopt\n \nin\n \nmanaging\n \nits\n \npeople.\n \nThey\n \nrepresent\n \nspecific\n \nguidelines\n \nto\n \nHR\n \nmanagers\n  \non\n \nvarious\n \nmatters\n \nconcerning\n \nemployment\n \nand\n \nstate\n \nthe\n \nintent\n \nof\n \nthe\n \norganization\n \non\n  \ndifferent\n \naspects\n \nof\n \nHuman\n \nResource\n \nmanagement\n \nsuch\n \nas\n \nrecruitment,\n \npromotion,\n  \ncompensation,\n \ntraining,\n \nselections\n \netc.\n \nThey\n \ntherefore\n \nserve\n \nas\n \na\n \nreference\n \npoint\n \nwhen\n  \nhuman

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Split the documents into smaller chunks for better RAG performance.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", "",",","."]
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"✅ Split {len(documents)} documents into {len(split_docs)} chunks.")

    # show an example chunk
    if split_docs:
        print("\n📄 Example chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs



In [11]:
chunks=split_documents(all_pdf_documents)
chunks

✅ Split 18 documents into 48 chunks.

📄 Example chunk:
Content: HUMAN  RESOURCE  POLICY     INTRODUCTION   
Human  resource  policies  are  continuing  guidelines  on  the  approach  of  which  an  
organization
  
intends
 
to
 
adopt
 
in
 
managing
 
its
 
peop...
Metadata: {'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'HR-Policy', 'source': '..\\data\\pdf\\HR-Policy (1).pdf', 'total_pages': 18, 'page': 0, 'page_label': '1', 'source_file': 'HR-Policy (1).pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'HR-Policy', 'source': '..\\data\\pdf\\HR-Policy (1).pdf', 'total_pages': 18, 'page': 0, 'page_label': '1', 'source_file': 'HR-Policy (1).pdf', 'file_type': 'pdf'}, page_content="HUMAN  RESOURCE  POLICY     INTRODUCTION   \nHuman  resource  policies  are  continuing  guidelines  on  the  approach  of  which  an  \norganization\n  \nintends\n \nto\n \nadopt\n \nin\n \nmanaging\n \nits\n \npeople.\n \nThey\n \nrepresent\n \nspecific\n \nguidelines\n \nto\n \nHR\n \nmanagers\n  \non\n \nvarious\n \nmatters\n \nconcerning\n \nemployment\n \nand\n \nstate\n \nthe\n \nintent\n \nof\n \nthe\n \norganization\n \non\n  \ndifferent\n \naspects\n \nof\n \nHuman\n \nResource\n \nmanagement\n \nsuch\n \nas\n \nrecruitment,\n \npromotion,\n  \ncompensation,\n \ntraining,\n \nselections\n \netc.\n \nThey\n \ntherefore\n \nserve\n \nas\n \na\n \nreference\n \npoint\n \nwhen\n  \nhuman

### EMBEDDINGS AND VECTORSTOREDB



In [14]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [19]:
from sentence_transformers import SentenceTransformer
from typing import List
import numpy as np

class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts.

        Args:
            texts: List of text strings to embed.
        Returns:
            numpy.ndarray: Embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


# Example usage
if __name__ == "__main__":
    embedding_manager = EmbeddingManager()
    texts = ["This is a test.", "Building a RAG chatbot with PDF documents."]
    embeddings = embedding_manager.generate_embeddings(texts)
    print("Embeddings shape:", embeddings.shape)


Loading embedding model: all-MiniLM-L6-v2


c:\Users\shilp\OneDrive\Desktop\rag-chatbot\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shilp\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed.

Model loaded successfully. Embedding dimension: 384
Generating embeddings for 2 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.72it/s]

Generated embeddings with shape: (2, 384)
Embeddings shape: (2, 384)


### VECTORSTORE

In [21]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "./data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()
    
    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )

            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [22]:
chunks

[Document(metadata={'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'HR-Policy', 'source': '..\\data\\pdf\\HR-Policy (1).pdf', 'total_pages': 18, 'page': 0, 'page_label': '1', 'source_file': 'HR-Policy (1).pdf', 'file_type': 'pdf'}, page_content="HUMAN  RESOURCE  POLICY     INTRODUCTION   \nHuman  resource  policies  are  continuing  guidelines  on  the  approach  of  which  an  \norganization\n  \nintends\n \nto\n \nadopt\n \nin\n \nmanaging\n \nits\n \npeople.\n \nThey\n \nrepresent\n \nspecific\n \nguidelines\n \nto\n \nHR\n \nmanagers\n  \non\n \nvarious\n \nmatters\n \nconcerning\n \nemployment\n \nand\n \nstate\n \nthe\n \nintent\n \nof\n \nthe\n \norganization\n \non\n  \ndifferent\n \naspects\n \nof\n \nHuman\n \nResource\n \nmanagement\n \nsuch\n \nas\n \nrecruitment,\n \npromotion,\n  \ncompensation,\n \ntraining,\n \nselections\n \netc.\n \nThey\n \ntherefore\n \nserve\n \nas\n \na\n \nreference\n \npoint\n \nwhen\n  \nhuman

In [23]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 48 texts...


Batches: 100%|██████████| 2/2 [00:01<00:00,  1.03it/s]


Generated embeddings with shape: (48, 384)
Adding 48 documents to vector store...
Successfully added 48 documents to vector store
Total documents in collection: 48


Retriever Pipeline From VectorStore

In [24]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [25]:
rag_retriever

In [27]:
rag_retriever.retrieve("What is HR policy")

Retrieving documents for query: 'What is HR policy'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 54.63it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


[{'id': 'doc_3e2be6ff_0',
  'content': "HUMAN  RESOURCE  POLICY     INTRODUCTION   \nHuman  resource  policies  are  continuing  guidelines  on  the  approach  of  which  an  \norganization\n  \nintends\n \nto\n \nadopt\n \nin\n \nmanaging\n \nits\n \npeople.\n \nThey\n \nrepresent\n \nspecific\n \nguidelines\n \nto\n \nHR\n \nmanagers\n  \non\n \nvarious\n \nmatters\n \nconcerning\n \nemployment\n \nand\n \nstate\n \nthe\n \nintent\n \nof\n \nthe\n \norganization\n \non\n  \ndifferent\n \naspects\n \nof\n \nHuman\n \nResource\n \nmanagement\n \nsuch\n \nas\n \nrecruitment,\n \npromotion,\n  \ncompensation,\n \ntraining,\n \nselections\n \netc.\n \nThey\n \ntherefore\n \nserve\n \nas\n \na\n \nreference\n \npoint\n \nwhen\n  \nhuman\n \nresources\n \nmanagement\n \npractices\n \nare\n \nbeing\n \ndeveloped\n \nor\n \nwhen\n \ndecisions\n \nare\n \nbeing\n  \nmade\n \nabout\n \nan\n \norganization's\n \nworkforce.\n \n A  good  HR  policy  provides  generalized  guidance  on  the  appro

In [28]:
rag_retriever.retrieve("When can one take leave")

Retrieving documents for query: 'When can one take leave'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 53.38it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_ccd08cfb_8',
  'content': 'if\n \nthe\n \nemployee\n \nhas\n \njoined\n \nthe\n \n \norganization\n \nin\n \nthe\n \nmiddle\n \nof\n \nthe\n \ncalendar\n \nyear,\n \nthen\n \nthe\n \nquantum\n \nof\n \nEarned\n \nLeaves\n \nwill\n \nbe\n \n \npro\n \nrata\n \nbasis.\n \n \nIn  case  of  new  joiners/employee  transfer,  following  rules  of  leave  credit  will  be  applicable:   \na)  Employee  joining/transferred  before  or  on  10th  of  the  month:  1.5  Leaves  \nb)\n \nEmployee\n \njoining/transferred\n \nbetween\n \n11th\n \nto\n \n18th\n \nof\n \nthe\n \nmonth:\n \n1\n \nLeave\n \nc)\n \nEmployee\n \njoining/transferred\n \nafter\n \n20th\n \nof\n \nthe\n \nmonth:\n \nNo\n \nCredit\n \n d)  We  give  0.5  leave  to  the  employee  on  his  birthday.   e)  For  Initial  Three  Months  No  Paid  Leave  will  be  allowed.   f)  In  a  Notice  Period  No  Leave  allow .   g)  All  leaves  shall  be  calculated  from  date  of  joining  of  the  employee.   h)  The  ma

###RAG Pipeline- VectorDB To LLM Output Generation

In [47]:
import os
from dotenv import load_dotenv
load_dotenv()

print(os.getenv("GROQ_API_KEY"))

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


In [48]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage

In [50]:
class GroqLLM:
    def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
        """
        Initialize Groq LLM
        
        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    

In [52]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

Initialized Groq LLM with model: gemma2-9b-it
Groq LLM initialized successfully!


In [53]:
# get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("leave policy for the employee")

Retrieving documents for query: 'leave policy for the employee'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.14s/it]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


[{'id': 'doc_73e83c25_7',
  'content': 'wages.\n \nAt\n \nRikalp\n \nwe\n \nfollow\n \nall\n \nthe\n \nstatutory\n \nnorms\n \nincluding\n \nemployee\n \nprovident\n \nfund,\n \nemployee\n  \nstate\n \ninsurance\n \nof\n \nthe\n \neligible\n \nemployees.\n \n \nLEAVE  POLICY   \nThe  leave  policy  is  created  to  give  employees  a  clear  picture  of  the  number  of  leaves  an   \nemployee\n \nis\n \nallowed\n \nto\n \ntake\n \nannually.\n \nFurthermore,\n \nit\n \nshould\n \nalso\n \nmention\n \npublic\n \nholidays.\n \n \nFurthermore,\n \nthe\n \nleave\n \npolicy\n \nmust\n \nalso\n \nmention\n \nthe\n \npercentage\n \nof\n \nsalary\n \ncut\n \nthat\n \nan\n \n \nemployee\n \nwill\n \nsee\n \nin\n \nthe\n \ncase\n \nof\n \nunpaid\n \nleaves\n \nor\n \nhalf-days.\n \nAll\n \nregular\n \nemployees\n \nare\n \nentitled\n \n \nfor\n \n18\n \nEarned\n \nLeaves\n \nin\n \na\n \ncalendar\n \nyear.\n \nHowever,\n \nif\n \nthe\n \nemployee\n \nhas\n \njoined\n \nthe\n \n \norganization\n

Integration Vectordb Context pipeline With LLM output

In [67]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")or "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"


# ✅ Updated model name
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile",
    temperature=0.1,
    max_tokens=1024
)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [68]:
answer=rag_simple("What is travel policy",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is travel policy'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.30it/s]


Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)
A travel policy is a set of guidelines that outlines the requirements and procedures for employees to follow when traveling for business-related work, including reimbursement of travel costs, proof requirements, and authorization authority.


Enhanced RAG Pipeline Features

In [69]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Leave policy", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'Leave policy'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 28.21it/s]

Generated embeddings with shape: (1, 384)
Retrieved 1 documents (after filtering)


Answer: Maternity leave of up to 60 days on full pay, along with other admissible allowances. For employees covered under ESIC, maternity leave will be provided by ESIC.
Sources: [{'source': 'HR-Policy (1).pdf', 'page': 5, 'score': 0.13112765550613403, 'preview': 'of  leave  title  along  with  the  maternity  leave  provided  that  the  total  leave  so  availed   does  not  exceed  60  days.  Maternity  leave  shall  be  leave  on  full  pay  together  with  any   \nother\n \nallowance\n \nthat\n \nmay\n \nbe\n \nadmissible.\n \n \nb)  Employee  who  are  covered  under ...'}]
Confidence: 0.13112765550613403
Context Preview: of  leave  title  along  with  the  maternity  leave  provided  that  the  total  leave  so  availed   does  not  exceed  60  days.  Maternity  leave  shall  be  leave  on  full  pay  together  with  any   
other
 
allowance
 
that
 
may
 
be
 
admissible.
 
 
b)  Employee  who  are  covered  under 


In [70]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("grooming and dress code policy of the company", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'grooming and dress code policy of the company'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.33it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
Customers  with  grievance   related  to  loan  or  against   staff  can  contact  to:   
B

RANCH  MANAGER/   REPORTING  MANAGER   
BM  Name  /  Reporting   Manager  Name:   Call  will  be  taken  between   09:30  am  to  6:00  pm  only   On  all  working  days    
If  the  complaint  /  dispute   in  not  redressed  within  a  period  of  15  days,  the   customer  may  approach  to   second  HR  Department  as   under:   
HR  Department    
If  the  complaint  /  dispute   in  not  redressed  within  a   period  of  one  month,  the   customer  may  approach  to   the  our  Vigilance  Officer   Email  Id  as  under:    
  GROOMING  &  DRESS  CODE  POLICY   
Gooming  and  Dress  code  guidelines  articulated  below  have  been  developed  in  order  to  
foster
 
a
 
professional
 
environment
 
in
 
value
 
and
 
culture.
 
It
 
shall
 
be
 
applicable
 
to
 
all
 
employees.
 
 
The  dress  code  policy  exists  to  make  working  more  professional  (formal  wear)  and  also,

promote  a  better  work  environment.  But,  a  good  dress  code  policy  is  a  set  of  guid